## Linear Regression with PyTorch

#### Part 2 of "PyTorch: Zero to GANs"

*This post is the second in a series of tutorials on building deep learning models with PyTorch, an open source neural networks library developed and maintained by Facebook. Check out the full series:*


1. [PyTorch Basics: Tensors & Gradients](https://jovian.ml/aakashns/01-pytorch-basics)
2. [Linear Regression & Gradient Descent](https://jovian.ml/aakashns/02-linear-regression)
3. [Image Classfication using Logistic Regression](https://jovian.ml/aakashns/03-logistic-regression) 
4. [Training Deep Neural Networks on a GPU](https://jovian.ml/aakashns/04-feedforward-nn)
5. [Image Classification using Convolutional Neural Networks](https://jovian.ml/aakashns/05-cifar10-cnn)
6. [Data Augmentation, Regularization and ResNets](https://jovian.ml/aakashns/05b-cifar10-resnet)
7. [Generating Images using Generative Adverserial Networks](https://jovian.ml/aakashns/06-mnist-gan)

Continuing where the [previous tutorial](https://jvn.io/aakashns/3143ceb92b4f4cbbb4f30e203580b77b) left off, we'll discuss one of the foundational  algorithms of machine learning in this post: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*.

We begin by importing Numpy and PyTorch:

In [1]:
import numpy as np
import torch

## Training data

The training data can be represented using 2 matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

We've separated the input and target variables, because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors as follows:

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [10]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True) # ramdom mainly from -1 to 1
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.8912,  0.5313,  0.8438],
        [-0.3793,  0.0491, -0.6526]], requires_grad=True)
tensor([ 0.1933, -0.7347], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [11]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [12]:
# Generate predictions
preds = model(inputs) # predict from random weight
print(preds)

tensor([[210.1295, -53.1960],
        [273.0478, -72.6970],
        [284.8597, -65.0070],
        [247.1598, -61.4573],
        [240.7550, -67.8757]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [13]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [15]:
diff = preds - targets
torch.sum(diff * diff) / diff.numel()
# off by about 170

tensor(28699.2129, grad_fn=<DivBackward0>)

You can see that there's a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [16]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor, and the `.numel` method returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [17]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(28699.2129, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by about 145 (square root of the loss 20834)*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. Also, the result is called the *loss*, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

In [18]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix, with the same dimensions.

In [19]:
# Gradients for weights
print(w)
print(w.grad)  # derivative

tensor([[ 1.8912,  0.5313,  0.8438],
        [-0.3793,  0.0491, -0.6526]], requires_grad=True)
tensor([[ 15125.7988,  14471.6875,   9302.4512],
        [-12992.6816, -14449.2266,  -8924.5322]])


The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the [slope](https://en.wikipedia.org/wiki/Slope) of the loss function w.r.t. the weights and biases.

If a gradient element is **positive**:
* **increasing** the element's value slightly will **increase** the loss.
* **decreasing** the element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/hFYoVgU.png)

If a gradient element is **negative**:
* **increasing** the element's value slightly will **decrease** the loss.
* **decreasing** the element's value slightly will **increase** the loss.

![negative=gradient](https://i.imgur.com/w3Wii7C.png)

The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model.

Before we proceed, we reset the gradients to zero by calling `.zero_()` method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call `.backward` on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.

In [20]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [21]:
w, b

(tensor([[ 1.8912,  0.5313,  0.8438],
         [-0.3793,  0.0491, -0.6526]], requires_grad=True),
 tensor([ 0.1933, -0.7347], requires_grad=True))

## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [22]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[210.1295, -53.1960],
        [273.0478, -72.6970],
        [284.8597, -65.0070],
        [247.1598, -61.4573],
        [240.7550, -67.8757]], grad_fn=<AddBackward0>)


Note that the predictions are same as before, since we haven't made any changes to our model. The same holds true for the loss and gradients.

In [23]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(28699.2129, grad_fn=<DivBackward0>)


In [24]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ 15125.7988,  14471.6875,   9302.4512],
        [-12992.6816, -14449.2266,  -8924.5322]])
tensor([ 174.9904, -156.0466])


Finally, we update the weights and biases using the gradients computed above.

In [25]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5 # if grad is pos, then move left
    b -= b.grad * 1e-5 # if grad is neg, then move right
    w.grad.zero_()
    b.grad.zero_()

A few things to note above:

* We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases. 

* We multiply the gradients with a really small number (`10^-5` in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the *learning rate* of the algorithm. 

* After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

Let's take a look at the new weights and biases.

In [26]:
print(w)
print(b)

tensor([[ 1.7399,  0.3866,  0.7508],
        [-0.2494,  0.1936, -0.5633]], requires_grad=True)
tensor([ 0.1915, -0.7331], requires_grad=True)


With the new weights and biases, the model should have lower loss.

In [27]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(19756.7402, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [34]:
# Train for 100 epochs
for i in range(10000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [35]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(0.5057, grad_fn=<DivBackward0>)


As you can see, the loss is now much lower than what we started out with. Let's look at the model's predictions and compare them with the targets.

In [36]:
# Predictions
preds

tensor([[ 57.1940,  70.1876],
        [ 82.1888, 100.7463],
        [118.6885, 132.9784],
        [ 21.0778,  37.0306],
        [101.9250, 119.1183]], grad_fn=<AddBackward0>)

In [37]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The prediction are now quite close to the target variables, and we can get even better results by training for a few more epochs. 

At this point, we can save our notebook and upload it to [Jovian.ml](https://www.jovian.ml) for future reference and sharing.

## Linear regression using PyTorch built-ins

The model and training process above were implemented using basic matrix operations. But since this such a common pattern , PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [38]:
import torch.nn as nn

As before, we represent the inputs and targets and matrices.

In [39]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [40]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

We are using 15 training examples this time, to illustrate how to work with large datasets in small batches.

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [41]:
from torch.utils.data import TensorDataset

In [45]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [47]:
from torch.utils.data import DataLoader

In [48]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)  # spilt data into small chunks to reduce compute heavy

The data loader is typically used in a `for-in` loop. Let's look at an example.

In [52]:
for xb, yb in train_dl:
    print('batch: ')
    print(xb)
    print(yb)
    break

batch: 
tensor([[ 91.,  88.,  64.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 91.,  88.,  64.]])
tensor([[ 81., 101.],
        [ 56.,  70.],
        [119., 133.],
        [103., 119.],
        [ 81., 101.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [53]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2212,  0.4771, -0.5485],
        [-0.1815,  0.1601,  0.5427]], requires_grad=True)
Parameter containing:
tensor([-0.0891, -0.2566], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [54]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2212,  0.4771, -0.5485],
         [-0.1815,  0.1601,  0.5427]], requires_grad=True),
 Parameter containing:
 tensor([-0.0891, -0.2566], requires_grad=True)]

We can use the model to generate predictions in the exact same way as before:

In [55]:
# Generate predictions
preds = model(inputs)  # initial prediction
preds

tensor([[ -7.8535,  20.5540],
        [-13.3338,  32.0449],
        [ 12.7909,  36.8792],
        [-22.4282,   8.1914],
        [ -7.9417,  40.5752],
        [ -7.8535,  20.5540],
        [-13.3338,  32.0449],
        [ 12.7909,  36.8792],
        [-22.4282,   8.1914],
        [ -7.9417,  40.5752],
        [ -7.8535,  20.5540],
        [-13.3338,  32.0449],
        [ 12.7909,  36.8792],
        [-22.4282,   8.1914],
        [ -7.9417,  40.5752]], grad_fn=<AddmmBackward>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [57]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [58]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [61]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(6195.7759, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD stands for `stochastic gradient descent`. It is called `stochastic` because samples are selected in batches (often with random shuffling) instead of as a single group.

In [64]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD`, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function `fit` which trains the model for a given number of epochs.

In [65]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb, yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update, and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement which prints the loss from the last batch of data for every 10th epoch, to track the progress of training. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [69]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/1000], Loss: 0.6766
Epoch [20/1000], Loss: 0.2541
Epoch [30/1000], Loss: 0.5734
Epoch [40/1000], Loss: 0.6100
Epoch [50/1000], Loss: 0.4086
Epoch [60/1000], Loss: 0.5401
Epoch [70/1000], Loss: 0.6486
Epoch [80/1000], Loss: 0.6587
Epoch [90/1000], Loss: 0.5880
Epoch [100/1000], Loss: 0.4006
Epoch [110/1000], Loss: 0.5304
Epoch [120/1000], Loss: 0.4012
Epoch [130/1000], Loss: 0.5859
Epoch [140/1000], Loss: 0.2749
Epoch [150/1000], Loss: 0.4373
Epoch [160/1000], Loss: 0.6644
Epoch [170/1000], Loss: 0.6611
Epoch [180/1000], Loss: 0.4355
Epoch [190/1000], Loss: 0.5842
Epoch [200/1000], Loss: 0.7081
Epoch [210/1000], Loss: 0.4451
Epoch [220/1000], Loss: 0.5087
Epoch [230/1000], Loss: 0.3862
Epoch [240/1000], Loss: 0.6444
Epoch [250/1000], Loss: 0.3489
Epoch [260/1000], Loss: 0.5181
Epoch [270/1000], Loss: 0.5299
Epoch [280/1000], Loss: 0.6593
Epoch [290/1000], Loss: 0.3837
Epoch [300/1000], Loss: 0.5091
Epoch [310/1000], Loss: 0.4940
Epoch [320/1000], Loss: 0.4298
Epoch [330/1000],

Let's generate predictions using our model and verify that they're close to our targets.

In [70]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.1662,  70.2684],
        [ 82.2368, 100.7020],
        [118.7229, 132.9695],
        [ 21.0969,  37.0245],
        [101.9389, 119.1348],
        [ 57.1662,  70.2684],
        [ 82.2368, 100.7020],
        [118.7229, 132.9695],
        [ 21.0969,  37.0245],
        [101.9389, 119.1348],
        [ 57.1662,  70.2684],
        [ 82.2368, 100.7020],
        [118.7229, 132.9695],
        [ 21.0969,  37.0245],
        [101.9389, 119.1348]], grad_fn=<AddmmBackward>)

In [71]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Indeed, the predictions are quite close to our targets, and now we have a fairly good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall and humidity in a region.

With this, we complete our discussion of linear regression in PyTorch, and we’re ready to move on to the next topic: *Logistic regression*.